In [1]:
# analysis notebook 

# see opt_fun.py for how these are calculated 
# see the individual files for the data 

In [2]:
import pandas as pd
import os
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from datetime import datetime
from IPython.display import display

In [3]:
from analysis_fun import *

In [4]:
import chart_studio
import chart_studio.plotly as py 
import chart_studio.tools as tls

username = "jnwagwu"
api_key = "119wNvkoTSGwHIxORQ85"

chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

# Import Data

In [5]:
def get_sub_dirs(path):
    sub_dirs = [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]
    return sub_dirs

def get_group_csv(group_model):
    group_model_path = os.path.join(opt_models_path, group_model )
    csv_path = os.path.join(group_model_path, "data.csv")
    df = pd.read_csv(csv_path, index_col="Unnamed: 0")
    return df

root_path = os.getcwd()
opt_models_path= os.path.join(root_path,"opt_models")

In [6]:
get_sub_dirs(opt_models_path)

['floor_2', 'roof_2', 'shgc_b1', 'window_u', 'wall_b1', 'floor_b1', 'roof_b1']

In [7]:
group_models = {
    "SHGC_true_ext": ["shgc_b1", "roof_b1", "wall_b1", "floor_b1"],
    "Ext_Surfaces": ["wall_b1", "floor_2", "roof_2"]
}


In [8]:
group_of_interest = "Ext_Surfaces"

def get_group(group_of_interest, example=False):
    group_data = {model: get_group_csv(model) for model in group_models[group_of_interest]}
    group_keys = list(group_data.keys())
    print(group_keys)
    if example:
        first_data = list(group_data.keys())[0]
        display(group_data[first_data])
        
    return group_keys, group_data, group_of_interest

# Exterior Surfaces

In [9]:
es_data = get_group("Ext_Surfaces", True)

['wall_b1', 'floor_2', 'roof_2']


,elect,hot,chill,elect_c,hot_c,chill_c,total,total_c,vals,perc_change
-98,110303.585613,49711.300809,104149.821091,20396.60,6784.88,12438.09,264164.707514,39619.57,0.070423,-98
-50,107453.878982,22565.561191,80634.937480,19910.75,3079.88,9629.82,210654.377653,32620.45,1.760563,-50
-25,107250.927219,20865.901685,79546.103348,19872.63,2847.90,9499.79,207662.932252,32220.32,2.640845,-25
0,107143.924137,19959.899623,78974.377892,19852.73,2724.24,9431.51,206078.201652,32008.48,3.521127,0
25,107077.444179,19398.951814,78620.299120,19839.96,2647.68,9389.23,205096.695113,31876.87,4.401408,25
50,107031.913910,19018.005027,78377.334576,19832.02,2595.69,9360.21,204427.253513,31787.92,5.281690,50
98,106975.964947,18548.616444,78073.438823,19821.49,2531.62,9323.92,203598.020213,31677.03,6.971831,98


In [10]:
four_compare(es_data[0], es_data[1], es_data[2])

Link  https://plotly.com/~jnwagwu/78/


In [11]:
correlations(es_data[1], es_data[2])

Link  https://plotly.com/~jnwagwu/80/


## SHGC_true_ext

In [12]:
shgc = get_group("SHGC_true_ext", False)
data = shgc
four_compare(data[0], data[1], data[2])
correlations(data[1], data[2])


['shgc_b1', 'roof_b1', 'wall_b1', 'floor_b1']


Link  https://plotly.com/~jnwagwu/82/


Link  https://plotly.com/~jnwagwu/84/
